In [1]:
import numpy as np
import sys
sys.path.append('..')
from lib.doctorailib.doctorailib import doctorai
from lib.doctorXAIlib.doctorXAIlib import doctorXAI
import pickle
from lib.embedding_utils import EmbeddingUtils, Embedding, EmbeddingType
from lib.utils_similarity import UtilsSimilarity
from lib.utils import Utils
from lib.semantic_enrichment import SemanticEnrichment, WindowStore
from lib.NotesCleaning import NotesCleaning
import gensim
import pandas as pd
import json
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as f
import numpy as np
import pandas as pd

## Load Data

In [13]:
# Load preliminary files
patient_admission_dictionary = pickle.load(open('../data/patient_admission_dictionary.pkl', 'rb'))
mapping_icd9_description = pickle.load(open("../data/icd9_mapping/mapping_icd9_description.pkl", "rb"))
description_dict = pickle.load(open("../data/mapping_relations/description.pkl",'rb'))
associated_morphology_dict = pickle.load(open("../data/mapping_relations/associated_morphology.pkl",'rb'))
due_to_dict = pickle.load(open("../data/mapping_relations/due_to.pkl",'rb'))
finding_site_dict = pickle.load(open("../data/mapping_relations/finding_site.pkl",'rb'))

In [ ]:
# Load dataset
df = pd.read_csv("../data/df_discharge.csv")

In [3]:
# Load BioWordVec embedding
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(
    '../data/embeddings/BioWordVec_PubMed_MIMICIII_d200.vec.bin',
    binary=True,
    limit=int(4E7)
)
embedding = Embedding(EmbeddingType.BIOWORDVEC, word2vec_model)

## Extract relevant ICD-9 with Doctor XAI

In [4]:
se = SemanticEnrichment()
subject_ids = [30575, 885]
notes_to_validate = se.get_notes_to_validate(subject_ids, df, patient_admission_dictionary, mapping_icd9_description)

In [11]:
notes_to_validate.head()

,subject_id,hadm_id,codes,description,notes
0,30575,114848,51881,Acute respiratory failure,Admission Date: [**2201-1-31**] ...
1,30575,114848,2920,Drug withdrawal,Admission Date: [**2201-1-31**] ...
2,30575,114848,7245,"Backache, unspecified",Admission Date: [**2201-1-31**] ...
3,30575,114848,07054,Chronic hepatitis C without mention of hepatic...,Admission Date: [**2201-1-31**] ...
4,30575,149751,V4364,Hip joint replacement,Admission Date: [**2203-6-26**] ...


## Extract Relevant Sentences

In [18]:
# Extract the relevant sentences. 
# The funcion split the sentences in higher than the threshold and lower than the threshold.
higher, lower = se.extract_relevant_sentences(notes_to_validate, 
                                threshold = 95, 
                                k = 1,
                                finding_site_dict = finding_site_dict,
                                due_to_dict = due_to_dict,
                                associated_morphology_dict = associated_morphology_dict,
                                description_dict = description_dict,
                                embedding = embedding)
higher = se.create_result_dataframe(higher)
lower = se.create_result_dataframe(lower)

In [20]:
higher.head()

,subject_ID,hadm_id,icd_9,relation_type,relation,similarity,extracted_substring,converted_string
0,30575,114848,51881,Finding_site,respiratory system structure,0.728156,"[for, hypercarbic, respiratory, acidosis, the,...",for hypercarbic\nrespiratory acidosis.\n.\nThe...
1,30575,114848,51881,Description,acute respiratory failure,0.810749,"[hypercarbic, respiratory, acidosis, the, pati...",hypercarbic\nrespiratory acidosis.\n.\nThe pat...
2,30575,114848,2920,Description,drug withdrawal,0.730278,"[stopped, ivda, in, aver, years, of, use, coca...",
3,30575,114848,7245,Finding_site,"back structure, including back of neck",0.916663,"[back, and, hit, the, back, of, his]",back and\nhit the back of his
4,30575,114848,7245,Description,backache,0.718363,"[his, chronic, back, hip, and, knee, pain, the...","his chronic back, hip, and knee pain. The pain..."
